Trying out dask to see if this is a better way to speed things up.

In [1]:
from ScreenPeptides import *
fp_dock_init()

PyRosetta-4 2020 [Rosetta PyRosetta4.conda.linux.CentOS.python37.Release 2020.10+release.46415fa3e9decb8b6e91a4e065c15543eb27a461 2020-03-05T09:09:24] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.linux.CentOS.python37.Release r248 2020.10+release.46415fa 46415fa3e9decb8b6e91a4e065c15543eb27a461 http://www.pyrosetta.org 2020-03-05T09:09:24
core.init: {0} command: PyRosetta -pep_refine -ex1 -ex2aro -use_input_sc -ignore_unrecognized_res -out:level 300 -multithreading:total_threads 1 -database /home/colin/anaconda3/envs/pyrosetta/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=-1004787300 seed_offset=0 real_seed=-1004787300 thread_index=0
basic.random.init_random_generator: {0} RandomGen

In [2]:
from dask.distributed import Client

In [3]:
client = Client(n_workers=24)
client

#this does not work when I use the extension, which is a bummer

Client Scheduler: tcp://127.0.0.1:46381 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 24 Cores: 24 Memory: 67.36 GB


In [4]:
%%writefile dask_docks.py

from pyrosetta.rosetta import *
from pyrosetta import *
import ScreenPeptides
import MutateInterface

def dask_testmut(pdb_filepath):
    from pyrosetta.rosetta.protocols.flexpep_docking import FlexPepDockingProtocol
    import ScreenPeptides
    import MutateInterface
    import random
    
    ScreenPeptides.fp_dock_init()

    pose = pose_from_pdb(pdb_filepath)
    position = random.choice(MutateInterface.interface_res(pose))
    aa = MutateInterface.choose_AA()
    toolbox.mutants.mutate_residue(pose, position, aa, pack_radius=8)
    
    fpdock = FlexPepDockingProtocol()
    fpdock.apply(pose)
    
    return pose.scores

def ddG_peptide(pdb_filepath, pdb_native_filepath):
    """
    Run FpDock and then calc ddG.
    
    First arg is path to pdb file. Second arg is path to native pdb file.
    """
    from pyrosetta.rosetta.protocols.flexpep_docking import FlexPepDockingProtocol
    import ScreenPeptides
    
    #pdb_filepath, pdb_native_filepath = args
    
    ScreenPeptides.fp_dock_init()

    pose = pose_from_pdb(pdb_filepath)
    native_pose = pose_from_pdb(pdb_native_filepath)
    
    fpdock = FlexPepDockingProtocol()
    fpdock.apply(pose)
    
    ddG = ScreenPeptides.ddG(native_pose, pose, 151)
    
    pose.scores['ddG'] = ddG
    pose.scores['fname'] = pose.pdb_info().name()
    
    return pose.scores

Overwriting dask_docks.py


Let's try docking some things.

In [5]:
%run MutateInterface.py
%run dask_docks.py
%run ScreenPeptides.py

In [8]:
from os import listdir
results = []
smbits = listdir('./smbits/')
for file in smbits:
    bitpath = './smbits/'+file
    nativepath = './smbits/native_test.pdb'
    r = client.submit(ddG_peptide, bitpath, nativepath)
    results.append(r)
#r = client.map(ddG_peptide, tasks, pure=False) #pure=False is very important for each process to get a unique seed

In [10]:
#g = client.gather(r)
for r in results:
    print(r)

<Future: status: error, key: ddG_peptide-86fbd25b2b089c3b52b3ba7e359cb0f3>
<Future: status: error, key: ddG_peptide-ec592ea1316bebb2252d2c1f5568d23f>
<Future: status: error, key: ddG_peptide-5cd6c94877055374f7deccdd1fa40997>
<Future: status: error, key: ddG_peptide-5e4a15d3c59faa42e2b8b296eb628254>
<Future: status: error, key: ddG_peptide-fba913356bc7e1c61549ab62db98ea3f>
<Future: status: error, key: ddG_peptide-afbd0be74be321ac808613559792e4ba>
<Future: status: error, key: ddG_peptide-acf9633b22f572246e512a41ec25ca56>
<Future: status: error, key: ddG_peptide-b7bd857cb4f9ba68f4621cd8ce1b590b>
<Future: status: error, key: ddG_peptide-a54873463c1593845dc1d2fb9502158e>


In [11]:
client.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/colin/anaconda3/envs/pyrosetta/lib/python3.7/site-packages/distributed/utils.py", line 662, in log_errors
    yield
  File "/home/colin/anaconda3/envs/pyrosetta/lib/python3.7/site-packages/distributed/client.py", line 1284, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/colin/anaconda3/envs/pyrosetta/lib/python3.7/site-packages/distributed/utils.py", line 662, in log_errors
    yield
  File "/home/colin/anaconda3/envs/pyrosetta/lib/python3.7/site-packages/distributed/client.py", line 1013, in _reconnect
    await self._close()
  File "/home/colin/anaconda3/envs/pyrosetta/lib/python3.7/site-packages/distributed/client.py", line 1284, in _close
    await gen.with_timeou